In [63]:
import tensorflow as tf

In [64]:
daily_sales_numbers=[21,22,-108,31,-1,32,34,31]


In [65]:
tf_dataset=tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

In [66]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [67]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [68]:
for sales in tf_dataset.take(3):
    print(sales)

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)


In [69]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [70]:
tf_dataset=tf_dataset.filter(lambda x:x>0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [71]:
tf_dataset=tf_dataset.map(lambda x:x*72)

In [72]:
tf_dataset=tf_dataset.shuffle(3)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)
#want to see what batches do then go to stackoverflow and research accordingly

1584
2304
1512
2232
2448
2232


In [73]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
2304
2232
1584
2232
2448


In [74]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[2232 2304]
[1584 2232]
[1512 2448]


# Creating a chain or Pipeline 

In [75]:
tf_dataset=tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset=tf_dataset.filter(lambda x:x>0).map(lambda y:y*72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 2232]
[2304 2448]
[1584 2232]


In [76]:
images_ds=tf.data.Dataset.list_files('Images/*/*',shuffle=False)
for file in images_ds.take(3):
    print(file.numpy)

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=string, numpy=b'Images\\Cat\\10 Facts about Cats - Companion Animals.jpg'>>
<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=string, numpy=b'Images\\Cat\\10_000_ Best Cat Photos \xc2\xb7 100_ Free.jpeg'>>
<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=string, numpy=b'Images\\Cat\\11 Stunning Persian Cats.jpg'>>


In [77]:
images_ds=images_ds.shuffle(25)
for file in images_ds.take(3):
    print(file.numpy())

b'Images\\Cat\\199_181 Cats Photos - Free & Royalty.jpg'
b'Images\\Cat\\How do cats get their stripes_ _ Live.jpg'
b'Images\\Cat\\Download Free Cats Stock Photos & HD Images.jpeg'


In [78]:
class_names=['Cat','Dog']

In [79]:
image_count=len(images_ds)
image_count

82

In [80]:
train_size=int(image_count*0.8)
train_ds=images_ds.take(train_size)
test_ds=images_ds.skip(train_size)


In [81]:
len(train_ds)

65

In [82]:
s='Images\\Cat\\10 Facts about Cats - Companion Animals.jpg'
s.split("\\")[-2]

'Cat'

# Fetching File Label from here

In [83]:
import os

In [84]:
def get_label(file_path):
    return tf.strings.split(file_path,os.path.sep)[-2]

In [85]:
get_label("Images\\Cat\\10 Facts about Cats - Companion Animals.jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'Cat'>

In [86]:
def process_img(file_path):
    label=get_label(file_path)
    img=tf.ip.read_file(file_path)
    img=tf.image.decode_jpeg(img)
    img=tf.image.resize(img,[128,128])
    return img,label

In [87]:
img, label = process_image("images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[:2]

NotFoundError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} NewRandomAccessFile failed to Create/Open: images\cat\20 Reasons Why Cats Make the Best Pets....jpg : The system cannot find the file specified.
; No such file or directory [Op:ReadFile]

In [88]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [89]:
for image, label in train_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[100.899414   164.42383     82.384766  ]
  [ 93.50488    141.72363     74.53711   ]
  [ 92.9834     123.7334      50.469727  ]
  ...
  [126.28223    176.01465    113.74805   ]
  [115.09766    165.81445     92.28418   ]
  [103.96387    165.2793      82.396484  ]]

 [[ 80.79297    142.63086     52.229492  ]
  [ 98.93848    156.78906     70.80469   ]
  [118.1875     153.86816     71.68555   ]
  ...
  [108.921875   162.39062     66.74219   ]
  [100.74805    159.57715     57.05957   ]
  [108.87598    163.62402     72.49121   ]]

 [[118.265625   168.91699     85.2207    ]
  [ 96.62695    157.12695     71.25195   ]
  [100.70801    157.02539     75.06738   ]
  ...
  [132.57812    162.72168     66.32617   ]
  [ 94.76465    156.38867     47.72168   ]
  [104.92676    161.78711     59.03711   ]]

 ...

 [[101.052734   189.76953     70.4209    ]
  [ 96.265625   139.92969     42.5625    ]
  [ 83.59277    117.677734    20.54004   ]
  ...
  [ 73.7207     130.33105     48.106445  ]
  

In [90]:
get_label("images\\Dog\\")

<tf.Tensor: shape=(), dtype=string, numpy=b'Dog'>

In [91]:
def scale(image, label):
    return image/255, label

In [92]:
train_ds = train_ds.map(scale)

In [93]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [1. 1. 1.]
****Label:  b'Cat'
****Image:  [0.00116422 0.00208333 0.00162377]
****Label:  b'Cat'
****Image:  [0.6486558  0.47610676 0.44081265]
****Label:  b'Cat'
****Image:  [0.15171503 0.11502691 0.08478794]
****Label:  b'Cat'
****Image:  [0.90588236 0.9254902  0.9490196 ]
****Label:  b'Cat'
